In [ ]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.5 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
from transformers import BertForSequenceClassification, BertTokenizer, get_linear_schedule_with_warmup
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
from tqdm import tqdm

In [ ]:
train_data = pd.read_csv('train.csv')
train_texts = train_data['text'].tolist()
train_scores = train_data['score'].values

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt')
train_scores = torch.tensor(train_scores, dtype=torch.float32)


(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)ert-large-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)arge-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)t-large-uncased/resolve/main/config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

In [ ]:
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_scores)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-large-uncased', num_labels=1)
model.train()

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,

In [ ]:
criterion = nn.MSELoss()
weight_decay = 0.1
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=weight_decay)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=len(train_dataloader) * 5)


In [ ]:
epochs = 10

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

for epoch in range(epochs):
    model.train()
    total_rmse = 0.0
    for batch in tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{epochs}'):
        optimizer.zero_grad()
        input_ids, attention_mask, targets = batch
        input_ids, attention_mask, targets = input_ids.to(device), attention_mask.to(device), targets.to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits.view(-1), targets)
        rmse = torch.sqrt(loss)
        rmse.backward()
        optimizer.step()
        scheduler.step()
        total_rmse += rmse.item()

    avg_rmse = total_rmse / len(train_dataloader)
    print(f'Epoch {epoch+1}/{epochs}, RMSE: {avg_rmse:.4f}')

Epoch 1/10: 100%|██████████| 750/750 [15:54<00:00,  1.27s/it]


Epoch 1/10, RMSE: 0.5830


Epoch 2/10: 100%|██████████| 750/750 [15:57<00:00,  1.28s/it]


Epoch 2/10, RMSE: 0.3686


Epoch 3/10: 100%|██████████| 750/750 [15:56<00:00,  1.28s/it]


...

Epoch 7/10: 100%|██████████| 750/750 [15:56<00:00,  1.28s/it]


Epoch 7/10, RMSE: 0.2759


Epoch 8/10: 100%|██████████| 750/750 [15:58<00:00,  1.28s/it]


Epoch 8/10, RMSE: 0.2714


Epoch 9/10:  77%|███████▋  | 581/750 [12:22<03:35,  1.28s/it]

In [ ]:
submission_data = pd.read_csv('sample_submission.csv')
submission_encodings = tokenizer(submission_data['text'].tolist(), truncation=True, padding=True, return_tensors='pt')
submission_dataset = TensorDataset(submission_encodings['input_ids'], submission_encodings['attention_mask'])
submission_dataloader = DataLoader(submission_dataset, batch_size=4)



In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
model.eval()
predicted_scores = []
for batch in tqdm(submission_dataloader, desc="Inferencing"):
    with torch.no_grad():
        input_ids, attention_mask = batch
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predicted_scores.extend(outputs.logits.view(-1).cpu().numpy())
submission_data['score'] = predicted_scores

In [ ]:
submission_data.to_csv('output.csv', index=False)